# Transformers 101

This notebook serves as an exploration of the transformer architecture (Vaswani et. al.) Here, we'll implement in native PyTorch the basic building blocks of the transformer and then put them all together so we have a model architecture to put into `../models`

In the process of putting this together (much like my other exploratory projects) I tried to limit viewing existing code online, and primarily used my notes (pdf attached for anyone interested) as a foundation for this work.

PS: The notes in this notebook are very much stream of thought, interpret accordingly

In [1]:
import torch
import math
import torch.nn as nn

## Encoder

In the first half of the notebook we'll develop all the pieces needed to implement the transformer encoder, then train and test it on some task. 

We want something with output dims: (sequence_length, output_dim)

In [2]:
def positional_embedding(input_tensor: torch.Tensor, output_dim: int, n=10000): 
    """
    Here, we implement the naive approach from the original 
    paper with the sin and cosine functions. 
    """
    P = torch.zeros((input_tensor.shape[-1], output_dim))
    indices = torch.arange(input_tensor.size(-1))
    i_values = torch.arange(int(output_dim/2))
    denominators = torch.float_power(n, 2*i_values/output_dim)
    P[:, 0::2] = torch.sin(indices.unsqueeze(1) / denominators.unsqueeze(0)) # start at 0, step by 2 sin for even nums
    P[:, 1::2] = torch.cos(indices.unsqueeze(1) / denominators.unsqueeze(0)) # start at 1, step by 2 cos for odd nums
    return P


In [3]:
a = torch.rand((2, 5))
output_dims = 3
positional_embedding(a, output_dims)

tensor([[ 0.0000,  1.0000,  0.0000],
        [ 0.8415,  0.5403,  0.8415],
        [ 0.9093, -0.4161,  0.9093],
        [ 0.1411, -0.9900,  0.1411],
        [-0.7568, -0.6536, -0.7568]])

In [4]:
def attention(x): 
    """
    Simple dot product based attention
    """
    query_layer, key_layer, value_layer = nn.Linear(x.shape[-1], x.shape[-1]), nn.Linear(x.shape[-1], x.shape[-1]), nn.Linear(x.shape[-1], x.shape[-1])
    query, key, value = query_layer(x), key_layer(x), value_layer(x)
    attention_weights  = torch.nn.Softmax(-1)(torch.tensordot(query, key, dims=1))
    return torch.sum(value * attention_weights)

In [5]:
x = torch.rand(1, 12)
attention(x)

tensor(0.1738, grad_fn=<SumBackward0>)

Just to emulate how it would be implemented, we write out the add norm function below. However in practice, this will be encompassed by each transformer sub module since each of them are followed by addition with residual and layer normalization. 

In [6]:
def add_norm(residual: torch.Tensor, hidden: torch.Tensor): 
    if residual.shape != hidden.shape: 
        raise ValueError("Shapes mismatch")
    else: 
        output = residual + hidden # element wise addition
        layer_norm = nn.LayerNorm([residual.shape[-2], residual.shape[-1]])
        return layer_norm(output)

In [7]:
# usage example: 

tensor_a = torch.rand([1, 5, 6]) # batch size, sequence length, embedding dimensions
tensor_b = torch.rand([1, 5, 6])
print(tensor_a)
print(tensor_b)
print(f"Final: {add_norm(tensor_a, tensor_b)}")

tensor([[[0.7833, 0.9884, 0.5462, 0.3524, 0.5592, 0.4115],
         [0.1378, 0.6331, 0.4987, 0.1650, 0.2863, 0.0467],
         [0.6652, 0.0613, 0.9855, 0.1929, 0.7510, 0.1926],
         [0.1679, 0.4150, 0.0477, 0.0384, 0.9646, 0.1112],
         [0.9000, 0.0346, 0.9401, 0.6464, 0.6922, 0.9834]]])
tensor([[[0.7859, 0.7694, 0.0275, 0.7968, 0.8284, 0.9946],
         [0.6791, 0.0083, 0.4875, 0.6828, 0.9286, 0.4735],
         [0.6746, 0.5864, 0.0128, 0.3205, 0.0690, 0.1453],
         [0.1133, 0.6379, 0.8655, 0.2424, 0.0102, 0.9089],
         [0.6790, 0.6481, 0.8799, 0.7385, 0.6040, 0.0767]]])
Final: tensor([[[ 1.3746,  1.8270, -1.0120,  0.3677,  0.9394,  0.9837],
         [-0.4288, -0.8495, -0.0230, -0.3549,  0.5254, -1.1403],
         [ 0.8246, -0.8344,  0.0060, -1.1566, -0.4215, -1.5772],
         [-1.7133,  0.1368, -0.1981, -1.7144, -0.0502,  0.0582],
         [ 1.3983, -0.7508,  1.9759,  0.9328,  0.7201,  0.1543]]],
       grad_fn=<NativeLayerNormBackward0>)


In [5]:
def scaled_dot_product_attention(q, k, d_k, mask = None):
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        # think of the last two dimensions of the attention tensor to be attention values for token-token pairs based on their indices (like a look up table of attention weights)
        # the diagonal will be the attention to the current token (self). The upper diagonal will contain attention with respect to tokens in the future and the lower in the past
        # therefore our 
        scores = scores.masked_fill(mask == 0, float('-inf'))
    return nn.Softmax(-1)(scores)

d_k and d_v are essentially hyperparameters that are fixed before training. This allows for the query and keys to have the same dimensionality, and for all 3 of them to have consistent dimensionality. In many transformer implementations, d_k and d_v are set to be the same for simplicity but this is not always the case.

In [7]:
def multihead_attention(k, q, v, d_k, d_v, d_model, num_heads):
    """
    Scaled Dot product based multi-head attention
    """
    # declare projection layers - assume all inputs have d_model size in the last dimension, and project to number of heads * d_k or d_v 
    query_layer, key_layer, value_layer = nn.Linear(d_model, num_heads* d_k), nn.Linear(d_model, num_heads* d_k), nn.Linear(d_model, num_heads*d_v)
    k_len, q_len, v_len, batch_size = k.size(1), q.size(1), v.size(1),  q.size(0)
    residual = q

    # in the following line we apply the linear projections and then reshape the outputs for multihead attention. 
    #The reshaping splits the last dimension of the linear layer's output into num_heads and d_k (or d_v for value). 
    # This creates multiple "heads" in the tensor, each with its own d_k (or d_v) dimension
    k, q, v = key_layer(k).view(batch_size, k_len,  num_heads, d_k), query_layer(q).view(batch_size, q_len,  num_heads, d_k), value_layer(v).view(batch_size, v_len,  num_heads, d_v)
    
    # we perform the following transpose so that the num heads dimension preceeds the seq length dimension. This way, each head can capture different information about the same sequence
    q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)
    attention = scaled_dot_product_attention(q, k, d_k)
    print(f"attention shape: {attention.shape}, value_shape: {v.shape}")
    output = torch.matmul(attention, v)
    print(output.shape)
    # following reshaping is done so that we can add our output to the residual 
    output = output.transpose(1, 2).contiguous().view(batch_size, q_len, -1)
    concatenated_projection = nn.Linear(num_heads * d_v, d_model, bias=False)

    output = concatenated_projection(output)
    output += residual

    print(residual.shape)
    layer_norm = nn.LayerNorm([residual.shape[-2], residual.shape[-1]])
    output = layer_norm(output)

    return output, attention

In [8]:
d_model = 512

# from the paper: To facilitate these residual connections, all sub-layers in the model, as well as the embedding layers, produce outputs of dimension d model 
k, q, v = torch.rand((1, 2, d_model)), torch.rand((1, 2, d_model)), torch.rand((1, 2, d_model))
d_k, d_v = 5, 5
num_heads = 4
print(v.shape)
out, attn = multihead_attention(k, q, v, d_k, d_v, d_model, num_heads)
print(out.shape, attn.shape)

torch.Size([1, 2, 512])
attention shape: torch.Size([1, 4, 2, 2]), value_shape: torch.Size([1, 4, 2, 5])
torch.Size([1, 4, 2, 5])
torch.Size([1, 2, 512])
torch.Size([1, 2, 512]) torch.Size([1, 4, 2, 2])


In [9]:
class PositionWiseFFN(nn.Module): 
    def __init__(self, d_model, d_ff, dropout) -> None:
        super(PositionWiseFFN, self).__init__()
        self.fc1 = nn.Sequential(nn.Linear(d_model, d_ff, bias=True),nn.ReLU())
        self.fc2 = nn.Linear(d_ff, d_model, bias=True)
        self.layer_norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        residual = x
        x = self.fc2(self.fc1(x))        
        return self.dropout(self.layer_norm(x+residual))


In [12]:
ffn = PositionWiseFFN(d_model, 2048, 0.1)
x = torch.rand((1, 2, d_model))
ffn(x)

tensor([[[-0.0000e+00, -1.9143e+00, -1.7751e-01,  ..., -7.1614e-01,
           1.7585e+00, -2.4166e-02],
         [-7.9474e-02, -1.4749e-03, -1.4262e+00,  ...,  9.2286e-01,
           1.7577e+00, -9.9777e-02]]], grad_fn=<MulBackward0>)

Now that we've implemented the lowest level building blocks of the transormer, below we put them together to build transformer blocks, encoder and decoder layers, and the complete transformer architecture. Now we try and condense everything to a more concise-less experimental implementation. 